In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn import ensemble
import seaborn as sns
import os
%matplotlib inline
# Constants
SEED = 123
FOLDS = 5
VERBOSE = 0
lookback_period = 30 # optional
gp_value = 0 # optional
biokemi = pd.read_csv('biokemi.csv', sep=';', encoding='latin-1')

In [ ]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
biokemi

In [ ]:
biokemi['ANALYSEKODE_MAIDS']

In [ ]:
biokemi.groupby('ANALYSEKODE_MAIDS')['REPLYUNIT'].nunique()

In [ ]:
# Group by 'analysis' and check if all corresponding 'unit' values are the same
result = biokemi.groupby('ANALYSEKODE_MAIDS')['REPLYUNIT'].nunique()

# Identify rows where 'unit' values for the same 'analysis' are different
rows_with_different_units = result[result > 1].index

# Display rows with different 'unit' values along with the units
if not rows_with_different_units.empty:
    print("Rows with different 'unit' values for the same 'analysis':")
    for analysis_value in rows_with_different_units:
        units_for_analysis = biokemi.loc[biokemi['ANALYSEKODE_MAIDS'] == analysis_value, 'REPLYUNIT'].unique()
        print(f"Analysis: {analysis_value}, Units: {', '.join(map(str, units_for_analysis))}")
else:
    print("All 'unit' values for the same 'analysis' are the same.")

In [ ]:
biokemi['ANALYSEKODE_MAIDS'] = biokemi['ANALYSEKODE_MAIDS'].str.replace('NA', 'Sodium')
biokemi['ANALYSEKODE_MAIDS'] = biokemi['ANALYSEKODE_MAIDS'].str.replace('nan', 'Sodium')

In [ ]:
biokemi['ANALYSEKODE_MAIDS'] = biokemi['ANALYSEKODE_MAIDS'].astype('category')

In [ ]:
biokemi

In [ ]:
biokemi.dropna(subset=['REPLY'], inplace=True)


In [ ]:
biokemi

In [ ]:
# Parse the PRVTAG_TID column into a new column named parsed_date
biokemi["parsed_date"] = pd.to_datetime(biokemi["PRVTAG_TID"], format="%d-%m-%Y %H:%M:%S", errors="coerce")

# If parsing failed, try to parse using dmy format for the date part only
mask = biokemi["parsed_date"].isna()
biokemi.loc[mask, "parsed_date"] = pd.to_datetime(
    biokemi.loc[mask, "PRVTAG_TID"],
    format="%d-%m-%Y",
    errors="coerce"
)

# Extract date from parsed_date and store it in a new column named date_only
biokemi["date_only"] = biokemi["parsed_date"].dt.date

biokemi["date_only"] = biokemi["date_only"] + pd.to_timedelta(gp_value, unit='d')

# Remove the parsed_date column if it's no longer needed
biokemi.drop("parsed_date", axis=1, inplace=True)


# Filter records from the beginning of 2010 to the end of 2020
biokemi = biokemi[(biokemi['date_only'] >= pd.to_datetime('2010-01-01').date()) & (biokemi['date_only'] <= pd.to_datetime('2020-12-31').date())]

# Display the filtered DataFrame
print(biokemi)

In [ ]:
# Replace ',' with '.' in the 'repl' column
biokemi['REPLY'] = biokemi['REPLY'].str.replace(',', '.')

In [ ]:
# biokemi['REPLY'] = biokemi['REPLY'].astype(float)

# Convert 'value' column to float, handling invalid entries by coercing to NaN
biokemi['REPLY'] = pd.to_numeric(biokemi['REPLY'], errors='coerce')

In [ ]:
# Conversion function
def convert_units(row):
    if row['ANALYSEKODE_MAIDS'] == 'TRANS':
        if row['REPLYUNIT'] == 'µmol/L':
            # No conversion needed for transferrin concentration in µmol/L
            return row['REPLY']
        elif row['REPLYUNIT'] == 'g/L':
            # Conversion from g/L to µmol/L using the molecular weight of transferrin
            molecular_weight = 79570.0  # molecular weight of transferrin in daltons
            conversion_factor = 1e6 / molecular_weight  # 1 g/L = 1e6 µmol/L (molecular weight in µg)
            return row['REPLY'] * conversion_factor
    else:
        return row['REPLY']

# Update the 'value' column with converted values for 'transferrin'
biokemi['REPLY'] = biokemi.apply(convert_units, axis=1)

In [ ]:
biokemi

In [ ]:
biokemi["INVESTIGATION_NAME"].unique()

In [ ]:
unique_analys_values = biokemi["ANALYSEKODE_MAIDS"].unique()
# Print all unique values one by one
for value in unique_analys_values:
    print(value)

In [ ]:
# Pivot the DataFrame to wide format
wide_bi = biokemi.pivot_table(index=['ID',"date_only"], columns='ANALYSEKODE_MAIDS', values='REPLY', aggfunc='mean')
# Reset the index if needed
wide_bi.reset_index(inplace=True)

# Display the wide-format DataFrame
print(wide_bi)

In [ ]:
wide_bi.shape

In [ ]:
wide_bi["BASO"]

In [ ]:
if all(col in wide_bi.columns for col in ["BASO", "BASOPO"]):
    wide_bi["BASO"] = np.nanmean(wide_bi[["BASO", "BASOPO"]], axis=1)
    wide_bi.drop(["BASOPO"], axis=1, inplace=True)
else:
    print("Either 'BASO' or 'BASOPO' not found in the DataFrame columns.")

In [ ]:
columns_to_process = [
    ("CAI", "CAIPOC"),
    ("CL", "CLPOC"),
    ("GLU", "GLUPOC"),
    ("HB", "HBPOC"),
    ("K", "KPOC"),
    ("LEU", "LEUPOC"),
    ("LYMFO", "LYMFOP"),
    ("MONO", "MONOPO")
]

for col, colpoc in columns_to_process:
    if all(c in wide_bi.columns for c in [col, colpoc]):
        wide_bi[col] = np.nanmean(wide_bi[[col, colpoc]], axis=1)
        wide_bi.drop([colpoc], axis=1, inplace=True)
    else:
        print(f"Either '{col}' or '{colpoc}' not found in the DataFrame columns.")


In [ ]:
wide_bi

### visualize biochemistry lab data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Select columns to visualize (excluding "ID" and "date")
columns_to_visualize = wide_bi.columns.difference(["ID", "date_only"])

# Set up subplots
fig, axes = plt.subplots(nrows=len(columns_to_visualize) // 3 + 1, ncols=3, figsize=(15, 5 * (len(columns_to_visualize) // 3 + 1)))

# Flatten axes for easy iteration
axes = axes.flatten()

# Loop through columns and plot histograms
for i, column in enumerate(columns_to_visualize):
    sns.histplot(wide_bi[column].dropna(), ax=axes[i], kde=True)
    axes[i].set_title(f'Distribution of {column}')

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
wide_bi.shape

In [ ]:
import pandas as pd
import numpy as np

# Select columns to filter (excluding "ID" and "date")
columns_to_filter = wide_bi.columns.difference(["ID", "date_only"])

# Define a custom function to filter negative and zero values and extreme outliers
def filter_negatives_and_extreme_outliers(column):
    # Convert negative values to NaN
    column[column <= 0] = np.nan
    
    # Calculate the first and third quartiles
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    
    # Calculate the interquartile range (IQR)
    IQR = Q3 - Q1
    
    # Define the lower and upper bounds for extreme outliers
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    
    # Convert extreme outliers to NaN
    column[(column < lower_bound) | (column > upper_bound)] = np.nan
    
    return column

# Apply the custom function to each selected column
wide_bi[columns_to_filter] = wide_bi[columns_to_filter].apply(filter_negatives_and_extreme_outliers)

# Display the modified DataFrame
print(wide_bi)


In [ ]:
# Specify the numerical features you want to scale
numerical_columns = wide_bi.select_dtypes(include=['float64', 'int64']).columns
has_negative_values = np.any(wide_bi[numerical_columns] < 0)

if has_negative_values:
    print("There are negative values in wide_bi.")
else:
    print("There are no negative values in wide_bi.")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Select columns to visualize (excluding "ID" and "date")
columns_to_visualize = wide_bi.columns.difference(["ID", "date_only"])

# Set up subplots
fig, axes = plt.subplots(nrows=len(columns_to_visualize) // 3 + 1, ncols=3, figsize=(15, 5 * (len(columns_to_visualize) // 3 + 1)))

# Flatten axes for easy iteration
axes = axes.flatten()

# Loop through columns and plot histograms
for i, column in enumerate(columns_to_visualize):
    sns.histplot(wide_bi[column].dropna(), ax=axes[i], kde=True)
    axes[i].set_title(f'Distribution of {column}')

# Adjust layout
plt.tight_layout()
plt.show()


### baseline variables for biochemical variables

In [ ]:
wide_bi.rename(columns={'date_only': 'date'}, inplace=True)

In [ ]:
wide_bi.describe()

In [ ]:
wide_bi.rename(columns={'SodiumPOC': 'Sodium'}, inplace=True)

In [ ]:
print(wide_bi)

#### Load LPR data (admission data already cleaned and filtered)

In [ ]:
LPR = pd.read_csv('LPR_filtered.csv') 

In [ ]:
len(LPR["ID"].unique())

In [ ]:
LPR["PATIENTTYPE"].unique()

#### Load bacteremia data (already cleaned and filtered)

In [ ]:
bacteremia = pd.read_csv('bacteremia_filtered.csv', encoding='latin-1')

In [ ]:
bacteremia.columns

In [ ]:
# Columns to exclude
exclude_columns = ["ID","TestDate","Sex","Age","BSIClass"]
bacteremia['allbac'] = bacteremia.apply(lambda row: ';'.join([col for col in bacteremia.columns if col not in exclude_columns and row[col]]), axis=1)

# Display the updated DataFrame
print(bacteremia)


In [ ]:
import numpy as np

in_columns1 = ["Staphylococcus aureus", "Staphylococcus aureus (MRSA)"]
bacteremia['Staphylococcus_aureus'] = bacteremia[in_columns1].any(axis=1)

in_columns2 = ["E. coli", "E. coli (ESBL positiv)"]
bacteremia['E.coli'] = bacteremia[in_columns2].any(axis=1)

in_columns3 = ~bacteremia.columns.isin(in_columns1 + in_columns2 + exclude_columns)
bacteremia['others'] = bacteremia.iloc[:, in_columns3].any(axis=1)

bacteremia["BSImulticlass"] = "noBSI"
for i in range(len(bacteremia)):
    if bacteremia.loc[i, 'Staphylococcus_aureus']:
        bacteremia.loc[i, "BSImulticlass"] = 'Staphylococcus_aureus'
    elif bacteremia.loc[i, 'E.coli']:
        bacteremia.loc[i, "BSImulticlass"] = 'E.coli'
    elif bacteremia.loc[i, 'others']:
        bacteremia.loc[i, "BSImulticlass"] = 'others'
  


In [ ]:
category_counts = bacteremia["BSImulticlass"].value_counts()
category_percentages = bacteremia["BSImulticlass"].value_counts(normalize=True) * 100

print("Category Counts:")
print(category_counts)

print("\nCategory Percentages:")
print(category_percentages)


In [ ]:
bacteremia.dtypes

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = bacteremia.copy()

# Convert the 'Date' column to datetime type
df['TestDate'] = pd.to_datetime(df['TestDate'])

# Specify the columns to exclude
excluded_columns = ['ID', 'TestDate', 'Sex', 'BSIClass', 'Age','allbac']

# Create time bins (1-year intervals)
time_bins = pd.date_range(start='2010-01-01', end='2020-12-31', freq='Y')

# Cut the 'Date' column into time bins
df['Time Bin'] = pd.cut(df['TestDate'], bins=time_bins, labels=time_bins[:-1])

# Initialize an empty dictionary to store results for each year
results_dict = {}

# Loop through each year
for i, year in enumerate(time_bins[:-1]):
    start_date = time_bins[i]
    end_date = time_bins[i + 1]
    
    # Filter data for the current year
    subset = df[(df['TestDate'] >= start_date) & (df['TestDate'] < end_date)]

    # Exclude specified columns
    subset = subset.drop(columns=excluded_columns, errors='ignore')

    # Count the cumulative occurrences of each bacteria for the current year
    bacteria_counts = subset.sum()

    # Find the top 10 most common bacteria for the current year
    top10_bacteria = bacteria_counts.nlargest(10).index.tolist()

    # Store the results for the current year in the dictionary
    results_dict[year] = {'Top 10 Bacteria': top10_bacteria, 'Bacteria Counts': bacteria_counts}

# Transpose the results for plotting
results_df = pd.DataFrame(results_dict).T

# Plot the results
plt.figure(figsize=(15, 8))

# Use seaborn for better color and style options
palette = sns.color_palette('husl', n_colors=15) 
markers = ['o', 's', '^', 'D', 'v', '>', '<', 'p', '*', 'h', '+', 'x', '|', '_', '.', ',']  

legend_names_added = set()

# Plot counts of top 10 bacteria for each year
for i, bacterium in enumerate(results_df['Top 10 Bacteria'].explode().unique()):
    marker = markers[i % len(markers)]  # Cycle through markers
    color = palette[i % len(palette)]  # Cycle through colors

    for year in results_df.index:
        # Check if the bacterium is in the top 10 for the current year
        if bacterium in results_df.loc[year, 'Top 10 Bacteria']:
            count = results_df.loc[year, 'Bacteria Counts'][bacterium]
            label = bacterium if bacterium not in legend_names_added else None
            plt.scatter([year], count, marker=marker, color=color, label=label)
            legend_names_added.add(bacterium)

# Plot counts of 'BSI' for each year on a logarithmic scale
bsi_counts = df[df['BSIClass'] == 'BSI'].groupby('Time Bin')['BSIClass'].count()
plt.scatter(bsi_counts.index, bsi_counts, marker='x', color='black', label='BSI')

plt.yscale('log')  # Set y-axis to logarithmic scale

plt.title('Top 10 most common bloodstream infections for each year (2010-2020)')
plt.xlabel('Year')
plt.ylabel('Count')
plt.legend(loc='upper right', bbox_to_anchor=(1.25, 1))
# Save the plot with high resolution
plt.savefig('top10infections.svg', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
print(bacteremia)

In [ ]:
LPR.columns

In [ ]:
selected_columns = ["ID", "INDDATO_DATO_TID", "UDDATO_DATO_TID"]
ADM = LPR[selected_columns] # Admission data
ADM = ADM.dropna(subset=['ID'])

ADM['ID'] = ADM['ID'].astype(int)
ADM['INDDATO_DATO_TID'] = pd.to_datetime(ADM['INDDATO_DATO_TID'])
ADM['UDDATO_DATO_TID'] = pd.to_datetime(ADM['UDDATO_DATO_TID'])

In [ ]:
ADM.rename(columns={'INDDATO_DATO_TID': 'IN_DATE'}, inplace=True)
ADM.rename(columns={'UDDATO_DATO_TID': 'OUT_DATE'}, inplace=True)

In [ ]:
ADM.shape

In [ ]:
max_adm_duration = 7

In [ ]:
# Filter rows where the difference between IN_DATE and OUT_DATE is less than or equal to 90 days
ADM_filtered = ADM[(ADM['OUT_DATE'] - ADM['IN_DATE']).dt.days <= max_adm_duration]

# Create a new column 'Date' and extend the rows per ID for the filtered DataFrame
ADM_filtered['Date'] = ADM_filtered.apply(lambda row: pd.date_range(row['IN_DATE'], row['OUT_DATE'], freq='D'), axis=1)

# Explode the 'Date' column to have one row per date
ADM_exploded = ADM_filtered.explode('Date')

# Reset the index if needed
ADM_exploded = ADM_exploded.reset_index(drop=True)

# Print the resulting DataFrame
print(ADM_exploded)

In [ ]:
ADM_exploded.rename(columns={'Date': 'date'}, inplace=True)

In [ ]:
ADM_filtered.shape

In [ ]:
selected_columns = ["ID", "TestDate", "Sex", "BSIClass", "Age",'allbac','BSImulticlass']
bacteremia_filtered = bacteremia[selected_columns]

In [ ]:
bacteremia_filtered.rename(columns={'TestDate': 'date'}, inplace=True)
bacteremia_filtered['date'] = pd.to_datetime(bacteremia_filtered['date'])

In [ ]:
bacteremia_filtered["date"] 

In [ ]:
wide_bi['date'] = pd.to_datetime(wide_bi['date'])

In [ ]:
# Check for duplicate keys in bacteremia_filtered
duplicate_keys = bacteremia_filtered[bacteremia_filtered.duplicated(subset=['ID', 'date'], keep=False)]
print(duplicate_keys)


In [ ]:
# Check for duplicate keys in wide_bi
duplicate_keys = wide_bi[wide_bi.duplicated(subset=['ID', 'date'], keep=False)]
print(duplicate_keys)


In [ ]:
merged_biokemi_bacteremia = pd.merge(wide_bi, bacteremia_filtered, on=['ID', 'date'], how='inner')

In [ ]:
# Check for duplicate keys in ADM_exploded
duplicate_keys = ADM_exploded[ADM_exploded.duplicated(subset=['ID', 'date'], keep=False)]
print(duplicate_keys)


In [ ]:
merged_biokemi_bacteremia

In [ ]:
ADM_exploded

In [ ]:
# Check for duplicate keys in merged_biokemi_bacteremia
duplicate_keys_merged = merged_biokemi_bacteremia[merged_biokemi_bacteremia.duplicated(subset=['ID', 'date'], keep=False)]
print(duplicate_keys_merged)


In [ ]:
# Remove duplicate keys in ADM_exploded
ADM_exploded_no_duplicates = ADM_exploded.drop_duplicates(subset=['ID', 'date'])

# Now, perform the left join again
merged_df = pd.merge(merged_biokemi_bacteremia, ADM_exploded_no_duplicates, on=['ID', 'date'], how='left')


In [ ]:
# Remove rows with NaN values in 'Column1'
merged_df = merged_df.dropna(subset=['BSIClass'])

In [ ]:
merged_df.shape

In [ ]:
wide_bi.columns

In [ ]:
# Specify columns you want to check for missing values
columns_to_check = wide_bi.columns

# Specify columns to exclude from the check
columns_to_exclude = ['ID', 'date']

# Remove excluded columns from the check
columns_to_check = [col for col in columns_to_check if col not in columns_to_exclude]

# Find rows where all specified columns have missing values
missing_rows = merged_df[merged_df[columns_to_check].isna().all(axis=1)]

In [ ]:
columns_to_check

In [ ]:
# Remove the identified rows from the DataFrame
merged_df = merged_df.drop(missing_rows.index)

In [ ]:
merged_df.shape

In [ ]:
# Calculate the ratios
merged_df['NEUTRO_to_LYMFO'] = np.where(merged_df['LYMFO'] != 0, merged_df['NEUTRO'] / merged_df['LYMFO'], np.nan)
merged_df['Platelet-to-lymphocyte'] = np.where(merged_df['LYMFO'] != 0, merged_df['THROM'] / merged_df['LYMFO'], np.nan)


In [ ]:
merged_df.columns

In [ ]:
merged_df

In [ ]:
columns_to_exclude = ['ID', 'IN_DATE',"OUT_DATE","date","Sex","BSIClass","Age","allbac"]

# Check for NaN values in all columns except those to exclude
merged_df_mask = merged_df.drop(columns=columns_to_exclude).isna().all(axis=1)

# Count the number of rows that meet the condition
count_rows_with_all_nan_except_some = merged_df_mask.sum()


In [ ]:
print(f"Number of rows with all NaN except for some columns: {count_rows_with_all_nan_except_some}")

In [ ]:
# Filter the DataFrame to exclude rows where all columns except for some are NaN
merged_df = merged_df[~merged_df_mask]

# Print the resulting DataFrame
merged_df

In [ ]:
merged_df.drop(["IN_DATE","OUT_DATE"], axis=1, inplace=True)

In [ ]:
merged_df.columns

In [ ]:
# Check for NaN values in the 'ID' column
nan_ids = merged_df['BSIClass'].isna().any()

In [ ]:
nan_ids

In [ ]:
merged_df.describe()

In [ ]:
from sklearn.model_selection import GroupKFold

# Extract features, labels, and groups
X = merged_df.drop('BSIClass', axis=1)  # Features
y = merged_df['BSIClass']  # Labels
groups = merged_df['ID']  # Groups

# Initialize GroupKFold with 5 splits
group_kfold = GroupKFold(n_splits=5)

# Split the data
for train_index, test_index in group_kfold.split(X, y, groups):
    train_set = merged_df.iloc[train_index]
    test_set = merged_df.iloc[test_index]

    # Display the result for each fold
    print("Training set:")
    print(train_set)
    print("\nTest set:")
    print(test_set)
    print("\n" + "="*40)  # Separating folds with a line


In [ ]:
# Count the occurrences of each class in the training set
train_class_counts = train_set['BSIClass'].value_counts()

# Count the occurrences of each class in the test set
test_class_counts = test_set['BSIClass'].value_counts()

# Display the proportions
print("Proportion of 'pos' to 'neg' in the training set:")
print("pos:", train_class_counts.get('BSI', 0) / len(train_set))
print("neg:", train_class_counts.get('noBSI', 0) / len(train_set))

print("\nProportion of 'pos' to 'neg' in the test set:")
print("pos:", test_class_counts.get('BSI', 0) / len(test_set))
print("neg:", test_class_counts.get('noBSI', 0) / len(test_set))


In [ ]:
# Check for common IDs between the training and test sets
common_ids = set(train_set['ID']).intersection(test_set['ID'])

# Display the result
if len(common_ids) == 0:
    print("No common IDs between the training and test sets.")
else:
    print("Common IDs between the training and test sets:", common_ids)


In [ ]:
len(common_ids)

In [ ]:
train_set.shape

In [ ]:
test_set.shape

In [ ]:
name_params = f'ADMper_{max_adm_duration}_multiclassBSI'

# Save the dataframes with the parameterized names
merged_df.to_csv(f'merged_df_{name_params}.csv', index=False)
train_set.to_csv(f'train_set_{name_params}.csv', index=False)
test_set.to_csv(f'test_set_{name_params}.csv', index=False)